In [1]:
pip install pandas geopandas matplotlib requests

You should consider upgrading via the '/Users/jeremy/.pyenv/versions/3.10.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
provinces = [{"code":"10","desc":"กรุงเทพมหานคร"},{"code":"11","desc":"สมุทรปราการ"},{"code":"12","desc":"นนทบุรี"},{"code":"13","desc":"ปทุมธานี"},{"code":"14","desc":"พระนครศรีอยุธยา"},{"code":"15","desc":"อ่างทอง"},{"code":"16","desc":"ลพบุรี"},{"code":"17","desc":"สิงห์บุรี"},{"code":"18","desc":"ชัยนาท"},{"code":"19","desc":"สระบุรี"},{"code":"20","desc":"ชลบุรี"},{"code":"21","desc":"ระยอง"},{"code":"22","desc":"จันทบุรี"},{"code":"23","desc":"ตราด"},{"code":"24","desc":"ฉะเชิงเทรา"},{"code":"25","desc":"ปราจีนบุรี"},{"code":"26","desc":"นครนายก"},{"code":"27","desc":"สระแก้ว"},{"code":"30","desc":"นครราชสีมา"},{"code":"31","desc":"บุรีรัมย์"},{"code":"32","desc":"สุรินทร์"},{"code":"33","desc":"ศรีสะเกษ"},{"code":"34","desc":"อุบลราชธานี"},{"code":"35","desc":"ยโสธร"},{"code":"36","desc":"ชัยภูมิ"},{"code":"37","desc":"อำนาจเจริญ"},{"code":"38","desc":"บึงกาฬ"},{"code":"39","desc":"หนองบัวลำภู"},{"code":"40","desc":"ขอนแก่น"},{"code":"41","desc":"อุดรธานี"},{"code":"42","desc":"เลย"},{"code":"43","desc":"หนองคาย"},{"code":"44","desc":"มหาสารคาม"},{"code":"45","desc":"ร้อยเอ็ด"},{"code":"46","desc":"กาฬสินธุ์"},{"code":"47","desc":"สกลนคร"},{"code":"48","desc":"นครพนม"},{"code":"49","desc":"มุกดาหาร"},{"code":"50","desc":"เชียงใหม่"},{"code":"51","desc":"ลำพูน"},{"code":"52","desc":"ลำปาง"},{"code":"53","desc":"อุตรดิตถ์"},{"code":"54","desc":"แพร่"},{"code":"55","desc":"น่าน"},{"code":"56","desc":"พะเยา"},{"code":"57","desc":"เชียงราย"},{"code":"58","desc":"แม่ฮ่องสอน"},{"code":"60","desc":"นครสวรรค์"},{"code":"61","desc":"อุทัยธานี"},{"code":"62","desc":"กำแพงเพชร"},{"code":"63","desc":"ตาก"},{"code":"64","desc":"สุโขทัย"},{"code":"65","desc":"พิษณุโลก"},{"code":"66","desc":"พิจิตร"},{"code":"67","desc":"เพชรบูรณ์"},{"code":"70","desc":"ราชบุรี"},{"code":"71","desc":"กาญจนบุรี"},{"code":"72","desc":"สุพรรณบุรี"},{"code":"73","desc":"นครปฐม"},{"code":"74","desc":"สมุทรสาคร"},{"code":"75","desc":"สมุทรสงคราม"},{"code":"76","desc":"เพชรบุรี"},{"code":"77","desc":"ประจวบคีรีขันธ์"},{"code":"80","desc":"นครศรีธรรมราช"},{"code":"81","desc":"กระบี่"},{"code":"82","desc":"พังงา"},{"code":"83","desc":"ภูเก็ต"},{"code":"84","desc":"สุราษฎร์ธานี"},{"code":"85","desc":"ระนอง"},{"code":"86","desc":"ชุมพร"},{"code":"90","desc":"สงขลา"},{"code":"91","desc":"สตูล"},{"code":"92","desc":"ตรัง"},{"code":"93","desc":"พัทลุง"},{"code":"94","desc":"ปัตตานี"},{"code":"95","desc":"ยะลา"},{"code":"96","desc":"นราธิวาส"}]

In [4]:
import requests
from requests.adapters import HTTPAdapter
from requests.exceptions import ConnectionError, Timeout
import json

timeout = 60  # Increase timeout to 5 seconds
retry_attempts = 3  # Retry the request up to 3 times
# Create a retry adapter with the desired number of retry attempts
retry_adapter = HTTPAdapter(max_retries=retry_attempts)
# Create a session and mount the retry adapter to it
session = requests.Session()

def get_districts(province_code):
    url = 'https://stat.bora.dopa.go.th/stat/statnew/connectSAPI/stat_forward.php?API=/api/stat/statcenter/v1/list?action=2'
    payload = {
        'cc':province_code
    }
    session.mount(url, retry_adapter)
    try:
        response = session.get(url,params=payload,timeout=timeout)
        json_response = response.json()
#     json_string = json.dumps(json_response, indent=2, ensure_ascii=False)
#     print(json_string)
        return json_response
    except (ConnectionError, Timeout) as e:
        print(f"Request failed: {e}")

def get_subdistricts(district_code):
    res = requests.get('https://stat.bora.dopa.go.th/stat/statnew/connectSAPI/stat_forward.php?API=/api/stat/statcenter/v1/list?action=614',{
        'rcode': district_code,
        'yymm': '6501' # need to send this stupid shit
    })
    resp = res.json()
    if type(resp) is dict:
        return None
    aa = res.json()[0]['aaCode']
    url = 'https://stat.bora.dopa.go.th/stat/statnew/connectSAPI/stat_forward.php?API=/api/stat/statcenter/v1/list?action=616'
    payload = {
        'yymm': 6501, 
        'rcode': district_code,
        'aa':aa
    }
    session.mount(url, retry_adapter)
    try:
        response = session.get(url,params=payload,timeout=timeout)
        json_response = response.json()
#     json_string = json.dumps(json_response, indent=2, ensure_ascii=False)
#     print(json_string)
        return json_response
    except (ConnectionError, Timeout) as e:
        print(f"Request failed: {e}")
        
def get_population_statistics_province_view(start, end):
    url = 'https://stat.bora.dopa.go.th/stat/statnew/connectSAPI/stat_forward.php?API=/api/statpophouse/v1/statpop/list?action=41'
    payload = {
        'yymmBegin': start, 
        'yymmEnd': end, 
        'statType':0, 
        'statSubType':999, 
        'subType':99
    }
    session.mount(url, retry_adapter)
    try:
        response = session.get(url,params=payload,timeout=timeout)
        json_response = response.json()
#     json_string = json.dumps(json_response, indent=2, ensure_ascii=False)
#     print(json_string)
        return json_response
    except (ConnectionError, Timeout) as e:
        print(f"Request failed: {e}")

def get_population_statistics_district_view(start, end,province_code,district_code):
    url = 'https://stat.bora.dopa.go.th/stat/statnew/connectSAPI/stat_forward.php?API=/api/statpophouse/v1/statpop/list?action=44'
    payload = {
        'yymmBegin': start, 
        'yymmEnd': end, 
        'statType':0, 
        'statSubType':999, 
        'subType':99,
        'cc':province_code,
        'rcode': district_code
    }
    session.mount(url, retry_adapter)
    try:
        response = session.get(url,params=payload,timeout=timeout)
        json_response = response.json()
#     json_string = json.dumps(json_response, indent=2, ensure_ascii=False)
#     print(json_string)
        return json_response
    except (ConnectionError, Timeout) as e:
        print(f"Request failed: {e}")

def get_population_statistics_subdistrict_view(start, end,province_code,district_code,subdistrict):
    url = 'https://stat.bora.dopa.go.th/stat/statnew/connectSAPI/stat_forward.php?API=/api/statpophouse/v1/statpop/list?action=45'
    payload = {
        'yymmBegin': start, 
        'yymmEnd': end, 
        'statType':0, 
        'statSubType':999, 
        'subType':99,
        'cc':province_code,
        'rcode': district_code,
        'tt': subdistrict
    }
    session.mount(url, retry_adapter)
    try:
        response = session.get(url,params=payload,timeout=timeout)
        json_response = response.json()
#     json_string = json.dumps(json_response, indent=2, ensure_ascii=False)
#     print(json_string)
        return json_response
    except (ConnectionError, Timeout) as e:
        print(f"Request failed: {e}")
  

In [6]:
bangkok_code = provinces[0]['code']
bkk_districts = get_districts(bangkok_code)
district_code = bkk_districts[0]['lsrcodecode']
bkk_district_1_subdistricts = get_subdistricts(district_code)
print(bkk_district_1_subdistricts)

[{'lscatmTTcode': 10010100, 'lscatmTTdesc': 'แขวงพระบรมมหาราชวัง'}, {'lscatmTTcode': 10010200, 'lscatmTTdesc': 'แขวงวังบูรพาภิรมย์'}, {'lscatmTTcode': 10010300, 'lscatmTTdesc': 'แขวงวัดราชบพิธ'}, {'lscatmTTcode': 10010400, 'lscatmTTdesc': 'แขวงสำราญราษฎร์'}, {'lscatmTTcode': 10010500, 'lscatmTTdesc': 'แขวงศาลเจ้าพ่อเสือ'}, {'lscatmTTcode': 10010600, 'lscatmTTdesc': 'แขวงเสาชิงช้า'}, {'lscatmTTcode': 10010700, 'lscatmTTdesc': 'แขวงบวรนิเวศ'}, {'lscatmTTcode': 10010800, 'lscatmTTdesc': 'แขวงตลาดยอด'}, {'lscatmTTcode': 10010900, 'lscatmTTdesc': 'แขวงชนะสงคราม'}, {'lscatmTTcode': 10011000, 'lscatmTTdesc': 'แขวงบ้านพานถม'}, {'lscatmTTcode': 10011100, 'lscatmTTdesc': 'แขวงบางขุนพรหม'}, {'lscatmTTcode': 10011200, 'lscatmTTdesc': 'แขวงวัดสามพระยา'}]


In [8]:
population_statistics_province_result = get_population_statistics_province_view(6412,6412)
print(len(population_statistics_province_result))

77


In [8]:
import pandas as pd

In [10]:

data = population_statistics_province_result
df = pd.DataFrame(data)
df = df.rename(columns={
    "lsccDesc": "province",
    "lssumtotMale": "male",
    "lssumtotFemale": "female",
    "lssumtotTot": "total"
})

df = df[["province", "male", "female", "total"]]
df.head(5)

,province,male,female,total
0,กรุงเทพมหานคร,2592292,2935702,5527994
1,จังหวัดสมุทรปราการ,645884,710565,1356449
2,จังหวัดนนทบุรี,599167,689470,1288637
3,จังหวัดปทุมธานี,563851,626209,1190060
4,จังหวัดพระนครศรีอยุธยา,394024,426488,820512


In [ ]:
  # การแสดง
  # 44 = ผลสำนักทะเบียน/อำเภอ
  # 45 = ตำบล / แขวง
  # 6501 = มกราคม 2565
  # 10 = กรุงเทพ
  # 1001 = เขตพระนคร
  # 100106 = แขวงเสาชิงช้า
data = get_population_statistics_district_view(6501,6512,10,1001)
df = pd.DataFrame(data)
df = df.rename(columns={
    "lsttDesc": "district",
    "lssumtotMale": "male",
    "lssumtotFemale": "female",
    "lssumtotTot": "total"
})

df = df[["district", "male", "female", "total"]]
df.head(5)

In [13]:
  # การแสดง
  # 44 = ผลสำนักทะเบียน/อำเภอ
  # 45 = ตำบล / แขวง
  # 6501 = มกราคม 2565
  # 10 = กรุงเทพ
  # 1001 = เขตพระนคร
  # 10010100 = แขวงพระบรมมหาราชวัง
data = get_population_statistics_subdistrict_view(6512,6512,10,1001,10010100)
print(data[0])
df = pd.DataFrame(data)
df = df.rename(columns={
    "lsttDesc": "subdistrict",
    "lssumtotMale": "male",
    "lssumtotFemale": "female",
    "lssumtotTot": "total"
})

df = df[["subdistrict", "male", "female", "total"]]
df.head(5)

{'lstrLevel': 0, 'lsregion': 0, 'lscc': 10, 'lsccDesc': 'กรุงเทพมหานคร', 'lsrcode': '1001', 'lsrcodeDesc': 'ท้องถิ่นเขตพระนคร', 'lsaa': 1, 'lsaaDesc': ' ', 'lstt': 1, 'lsttDesc': 'แขวงพระบรมมหาราชวัง', 'lsmm': 0, 'lsmmDesc': 'พระบรมมหาราชวัง', 'lsyymm': 6512, 'lssumtotMale': 1845, 'lssumtotFemale': 1292, 'lssumtotTot': 3137}


,subdistrict,male,female,total
0,แขวงพระบรมมหาราชวัง,1845,1292,3137


In [6]:
district_results = []
subdistrict_results = []
for province in provinces:
    province_code = province['code']
    districts = get_districts(province_code)
    for district in districts:
        district_code = district['lsrcodecode']
        data = get_population_statistics_district_view(6512,6512,province_code,district_code)
        if type(data) is not dict:
            district_results = district_results + data
        print('province_code: {0} district_code: {1}'.format(province_code,district_code))  
        subdistricts = get_subdistricts(district_code)
        if subdistricts is None:
            continue
        for subdistrict in subdistricts: 
            subdistrict_code = subdistrict['lscatmTTcode']
            subdistricts_data = get_population_statistics_subdistrict_view(6512,6512,province_code,district_code,subdistrict_code)
            if type(subdistricts_data) is not dict:
                subdistrict_results = subdistrict_results + subdistricts_data
print(len(district_results))   
print(len(subdistrict_results))   

province_code: 10 district_code: 1001
province_code: 10 district_code: 1002
province_code: 10 district_code: 1003
province_code: 10 district_code: 1004
province_code: 10 district_code: 1005
province_code: 10 district_code: 1006
province_code: 10 district_code: 1007
province_code: 10 district_code: 1008
province_code: 10 district_code: 1009
province_code: 10 district_code: 1010
province_code: 10 district_code: 1011
province_code: 10 district_code: 1012
province_code: 10 district_code: 1013
province_code: 10 district_code: 1014
province_code: 10 district_code: 1015
province_code: 10 district_code: 1016
province_code: 10 district_code: 1017
province_code: 10 district_code: 1018
province_code: 10 district_code: 1019
province_code: 10 district_code: 1020
province_code: 10 district_code: 1021
province_code: 10 district_code: 1022
province_code: 10 district_code: 1023
province_code: 10 district_code: 1024
province_code: 10 district_code: 1025
province_code: 10 district_code: 1026
province_cod

In [11]:
data = subdistrict_results
df = pd.DataFrame(data)
df = df.rename(columns={
    "lsccDesc": "province",
    "lsttDesc": "district",
    "lssumtotMale": "male",
    "lssumtotFemale": "female",
    "lssumtotTot": "total"
})
df.to_csv('subdistrict_population.csv')
df.head(5)

,lstrLevel,lsregion,lscc,province,lsrcode,lsrcodeDesc,lsaa,lsaaDesc,lstt,district,lsmm,lsmmDesc,lsyymm,male,female,total
0,0,0,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,1,,1,แขวงพระบรมมหาราชวัง,0,พระบรมมหาราชวัง,6512,1845,1292,3137
1,0,0,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,1,,2,แขวงวังบูรพาภิรมย์,0,วังบูรพาภิรมย์,6512,4385,4407,8792
2,0,0,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,1,,3,แขวงวัดราชบพิธ,0,วัดราชบพิธ,6512,1310,1603,2913
3,0,0,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,1,,4,แขวงสำราญราษฎร์,0,สำราญราษฎร์,6512,1385,1327,2712
4,0,0,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,1,,5,แขวงศาลเจ้าพ่อเสือ,0,ศาลเจ้าพ่อเสือ,6512,1174,1573,2747


In [ ]:
data = district_results
df = pd.DataFrame(data)
df = df.rename(columns={
    "lsccDesc": "province",
    "lsttDesc": "district",
    "lssumtotMale": "male",
    "lssumtotFemale": "female",
    "lssumtotTot": "total"
})
df.to_csv('district_population.csv')
df.head(5)